# Проект <span style="color:red">Я</span>ндекс.Поток

## Описание проекта

### Постановка задачи + Анализ источников

Разработать программу, которая по заданному треку подбирает наиболее подходящие по жанру треки и выводит результат в порядке убывания "схожести"


https://www.shazam.com/ru/charts/genre/world/dance

Цели:
1. <p style="color:green">Найти всевозможные ссылки на агрегаторы музыкальные либо на поисковики (shazam, zaycev)</p>
2. Концепция: по названию ищем жанр(-ы), далее находим похожие по жанру треки (по популярности) + язык + треки похожих исполнителей + треки того же года  -> на первой сотне пытаемся найти пересечения
Если они, то выводим список пересечений. Если нет, то ...

Получив требуемые ресурсы, составить словарь синонимов для жанров, чтобы не было проблем с составленим запросов


## Подготовка

###  Подключаем библиотеки и устанавливаем рабочую директорию

In [7]:
import pandas as pd
from pandas import read_csv

In [8]:
import os
os.chdir(os.getcwd())

### Подгружаем данные по ТОП-200 

In [11]:
df_rus_200 = read_csv("data/ShazamTopRus200.csv", skiprows = 2, on_bad_lines='skip')
df_rus_200.head()

,Rank,Artist,Title
0,1,DJ Smash & NIVESTA,Позвони
1,2,Markul & Тося Чайкина,Стрелы
2,3,Bakr,Вредина (Mbts Remix)
3,4,Miyagi & Эндшпиль,Silhouette
4,5,Mario Fresh & Renvtø,Necesar


In [12]:
df_rus_200.shape

(200, 3)

In [13]:
df_world_200 = read_csv("data/ShazamTopWorld200.csv", skiprows = 2, on_bad_lines='skip')
df_world_200.head()

,Rank,Artist,Title
0,1,JIN,The Astronaut
1,2,Sam Smith & Kim Petras,Unholy
2,3,Rihanna,Lift Me Up (From Black Panther: Wakanda Foreve...
3,4,Oliver Tree & Robin Schulz,Miss You
4,5,Rema,Calm Down


In [14]:
df_world_200.shape

(199, 3)

In [22]:
not df_world_200.loc[df_world_200['Title']=='Miss You'].empty

True

## Реализация

In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import requests
import os
os.chdir(os.getcwd())

### Databases

In [2]:
# Top-200
df_rus_top = read_csv("data/ShazamRusTop200.csv", skiprows = 2, on_bad_lines='skip')
df_world_top = read_csv("data/ShazamWorldTop200.csv", skiprows = 2, on_bad_lines='skip')

# Pop
df_rus_pop = read_csv("data/ShazamRusPop.csv", skiprows = 2, on_bad_lines='skip')
df_world_pop = read_csv("data/ShazamWorldPop.csv", skiprows = 2, on_bad_lines='skip')

# Hip-Hop
df_rus_hiphop = read_csv("data/ShazamRusHipHop.csv", skiprows = 2, on_bad_lines='skip')
df_world_hiphop = read_csv("data/ShazamWorldHipHop.csv", skiprows = 2, on_bad_lines='skip')

# Dance
df_rus_dance = read_csv("data/ShazamRusDance.csv", skiprows = 2, on_bad_lines='skip')
df_world_dance = read_csv("data/ShazamWorldDance.csv", skiprows = 2, on_bad_lines='skip')

In [3]:
# MAIN DATABASE

music_dict = {
    "Top200": [
        {
            "path": "data/ShazamRusTop200.csv", 
            "df": df_rus_top,
            "ref": "https://www.shazam.com/services/charts/csv/top-200/russia/"
        }, 
        {
            "path": "data/ShazamWorldTop200.csv", 
            "df": df_world_top,
            "ref": "https://www.shazam.com/services/charts/csv/top-200/world/"
        }],
    "Pop":[
        {
            "path": "data/ShazamRusPop.csv", 
            "df": df_rus_pop,
            "ref": "https://www.shazam.com/services/charts/csv/genre/russia/pop"
        }, 
        {
            "path":"data/ShazamWorldPop.csv", 
            "df": df_world_pop,
            "ref": "https://www.shazam.com/services/charts/csv/genre/world/pop"
        }], 
    "Hip-Hop":[
        {
            "path": "data/ShazamRusHipHop.csv", 
            "df": df_rus_hiphop,
            "ref": "https://www.shazam.com/services/charts/csv/genre/russia/hip-hop-rap"
        }, 
        {
            "path": "data/ShazamWorldHipHop.csv", 
            "df": df_world_hiphop,
            "ref": "https://www.shazam.com/services/charts/csv/genre/world/hip-hop-rap"
        }], 
    "Dance": [
        {
            "path": "data/ShazamRusDance.csv", 
            "df": df_rus_dance,
            "ref": "https://www.shazam.com/services/charts/csv/genre/russia/dance"
        }, 
        {
            "path": "data/ShazamWorldDance.csv", 
            "df": df_world_dance,
            "ref": "https://www.shazam.com/services/charts/csv/genre/world/dance"
        }]
        }

### Функция подборки музыки getNextTrack

In [4]:
def getNextTrack(title: str, artist = "Noname", track_num = 1) -> list:
    """ Функция поиска следующего/-их для воспроизведения трека/-ов """
    
    search_res = []
    
    # поиск подходящих баз данных из числа имеющихся
    for key, value in music_dict.items():
        for data in value:
            df = data["df"]
            if not df.loc[df['Title'] == title].empty:
                search_res.append([key, df])
                
    # если подходящих не нашлось
    if not search_res:
        if artist == "Noname":
            variants = music_dict["Top200"][0]["df"]["Title"]
            return variants.sample(n=track_num, random_state=np.random.randint(1, 1000)).values[:track_num].tolist()
        else:
            pass

    # если мы нашли хотя бы одну подходящую базу данных
    variants = []
    for key, df in search_res:
        for track in df["Title"].values:
            variants.append([key, track])
            
    variants = np.array(variants)
    np.random.shuffle(variants)
    
    return variants[:track_num].tolist()
        
    

__Проверка__

In [5]:
getNextTrack("asdfghj", track_num = 5)

["Don't Stop the Party",
 '1, 2, 3 (feat. De La Ghetto & Jason Derulo)',
 'Угонщица 2023',
 'Снегири',
 'Мечтали']

In [6]:
getNextTrack("Miss You", track_num = 5)

[['Top200', 'Marea (We’ve Lost Dancing)'],
 ['Top200', 'Collide (feat. Tyga) [Sped Up Remix]'],
 ['Dance', 'When Love Takes Over (feat. Kelly Rowland) [Mixed]'],
 ['Top200', "I'm Good (Blue)"],
 ['Dance', 'Pump It Louder']]

### Функция обновления данных песен updateSongs

In [7]:
def updateSongs():   
    from time import sleep
    print(f"Выберите базу для обновления (0-{len(music_dict)}):")
    for i, key in enumerate(music_dict):
        print(f"{i}: {key}")
    print(f"{len(music_dict)}: Обновить все")
    choice = int(input("\nВаш выбор:\n"))
    
    if choice not in range(len(music_dict) + 1):
        raise ValueError('Некорректный выбор')
    elif choice < len(music_dict):
        name = list(music_dict.keys())[choice]
        print(f"Обновление баз {name} запущено")
        
        for indx in range(len(music_dict[name])):
            # Вводим ссылку на скачивание csv
            URL = music_dict[name][indx]["ref"]

            # Перезаписываем имеющийся csv файл на новый
            try:
                response = requests.get(URL)
            except:
                print(f"База номер {indx} в {name} пропущена...")
                continue
            open(music_dict[name][indx]["path"], "wb").write(response.content)

            # Обновляем DataFrame
            music_dict[name][indx]["df"] = read_csv(music_dict[name][indx]["path"], skiprows = 2, on_bad_lines='skip')
        
        print(f"Обновление баз {name} завершено")
    else:
        for name in music_dict:
            print(f"Обновление баз {name} запущено")
            
            for indx in range(len(music_dict[name])):
                # Вводим ссылку на скачивание csv
                URL = music_dict[name][indx]["ref"]

                # Перезаписываем имеющийся csv файл на новый
                try:
                    response = requests.get(URL)
                except:
                    print(f"База номер {indx} в {name} пропущена...")
                    continue
                open(music_dict[name][indx]["path"], "wb").write(response.content)

                # Обновляем DataFrame
                music_dict[name][indx]["df"] = read_csv(music_dict[name][indx]["path"], skiprows = 2, on_bad_lines='skip')
            sleep(0.5)
            print(f"Обновление баз {name} завершено")
            sleep(1)
        

__Проверка__

In [8]:
updateSongs()

Выберите базу для обновления (0-4):
0: Top200
1: Pop
2: Hip-Hop
3: Dance
4: Обновить все



Ваш выбор:
 4


Обновление баз Top200 запущено
Обновление баз Top200 завершено
Обновление баз Pop запущено
Обновление баз Pop завершено
Обновление баз Hip-Hop запущено
Обновление баз Hip-Hop завершено
Обновление баз Dance запущено
Обновление баз Dance завершено


### Парсер новинок(обновление рейтингов + добавление треков)

####  Read ownChart

In [9]:
def getOwnChart():
    return pd.read_csv("data/ownChart.csv")


In [76]:
df = getOwnChart()
df.head()

,Artist,Title,Views (per 1k)
0,Markul & Тося Чайкина,Стрелы,112.2


In [75]:
# df.to_csv("data/ownChart.csv", index=False)

#### Add new Track

##### Prepare

In [13]:
req = {'q': 'эклер heroinwater' + ' genius'}  
r = requests.get('https://www.google.com/search', params=req)
print(r.url)

https://www.google.com/search?q=%D1%8D%D0%BA%D0%BB%D0%B5%D1%80+heroinwater+genius


In [14]:
soup = BeautifulSoup(r.content, "html.parser")
info = ""
target = soup.findAll("a", limit =20)

In [15]:
for a_tag in target:
    href = a_tag.attrs.get("href")
    if href == "" or href is None or ("/url?q=" not in href):
        continue
    info = href[href.find("=") + 1:href.find("&")]
    break
    

In [16]:
info

'https://genius.com/Heronwater-ekler-lyrics'

In [31]:
soup_info = BeautifulSoup(requests.get(info).content, "html.parser")
target = []
for sp in soup_info.find_all('span', limit=15):
    if sp.text:
        target.append(sp.text)

In [37]:
song_title = target[4]
song_artist = target[5]
song_views = float(sorted(target)[0][:-1])

print(f"Title: {song_title}")
print(f"Artist: {song_artist}")
print(f"Views: {song_views}k")

Title: Эклер (Ekler)
Artist: Heronwater
Views: 11.1k


##### Result

In [83]:
from bs4 import BeautifulSoup
def searchNewTrack(song_info: str):
    req = {'q': song_info + ' genius'}  
    r = requests.get('https://www.google.com/search', params=req)

    soup = BeautifulSoup(r.content, "html.parser")
    info = ""
    search_res = soup.findAll("a", limit =20)
    for a_tag in search_res:
        href = a_tag.attrs.get("href")
        if href == "" or href is None or ("/url?q=" not in href):
            continue
        info = href[href.find("=") + 1:href.find("&")]
        break

    if not info:
        print("По вашему запросу не удалось найти песню. Повторите попытку с другими ключевыми словами")
    soup_info = BeautifulSoup(requests.get(info).content, "html.parser")
    target = []
    for sp in soup_info.find_all('span', limit=15):
        if sp.text:
            target.append(sp.text)
    song_title = target[4]
    song_artist = target[5]
    song_views = target[-1]


    print("По вашему запросу была найдена следующая песня:")
    print(f"Title: {song_title}")
    print(f"Artist: {song_artist}\n")
    isValid = input("Добавлять найденный трек в базу: да или нет\n")
    if isValid.lower() == "да":
        if 'M' in song_views:
            song_views = float(song_views[:-1]) * 1000
        else:
            song_views = float(song_views[:-1])
        df = getOwnChart()
        df = df.append({'Artist': song_artist, 'Title': song_title, 'Views (per 1k)': song_views}, ignore_index=True)
        df = df.drop_duplicates()
        df.to_csv("data/ownChart.csv", index=False)
        print("База успешно обновлена")
    return

##### Test

In [78]:
df = getOwnChart()
df.head()

,Artist,Title,Views (per 1k)
0,Markul & Тося Чайкина,Стрелы,112.2


In [79]:
searchNewTrack("эклер heroinwater")

По вашему запросу была найдена следующая песня:
Title: Эклер (Ekler)
Artist: Heronwater



Добавлять найденный трек в базу: да или нет
 да


In [80]:
df = getOwnChart()
df.head()

,Artist,Title,Views (per 1k)
0,Markul & Тося Чайкина,Стрелы,112.2
1,Heronwater,Эклер (Ekler),11.1


In [81]:
searchNewTrack("morgenstern кадилак")

По вашему запросу была найдена следующая песня:
Title: Cadillac
Artist: MORGENSHTERN & Элджей (Eldzhey)



Добавлять найденный трек в базу: да или нет
 да


In [82]:
df = getOwnChart()
df.head()

,Artist,Title,Views (per 1k)
0,Markul & Тося Чайкина,Стрелы,112.2
1,Heronwater,Эклер (Ekler),11.1
2,MORGENSHTERN & Элджей (Eldzhey),Cadillac,2000.0
